In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4306ae9eb0ca074cd0a5beb6557d750d0e5dd0175275092b5d4173e1774c6e88
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, models, losses, InputExample
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer
import torch
from scipy.stats import spearmanr
from torch.utils.data import DataLoader

In [3]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [4]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [5]:
df_train_path = '/content/vists_train.csv'
df_train = pd.read_csv(df_train_path)
df_train

,Unnamed: 0,year,sentence1,sentence2,score
0,0,2012test,Một người phụ nữ đang gọt khoai tây.,Một người đang cắt khoai tây.,2.40
1,1,2012test,Một người đang thái khoai tây.,Một người phụ nữ đang gọt khoai tây.,2.20
2,2,2012test,Một người đàn ông đang đi xe đạp.,Một con khỉ đang đạp xe.,2.00
3,3,2012test,Một người đang cưỡi ngựa.,Một cô gái đang cưỡi ngựa.,2.60
4,4,2012test,Hai thằng nhóc đang lái xe.,Hai vịnh đang nhảy múa.,0.60
...,...,...,...,...,...
4995,4995,2014,Thủ đô Indonesia cầu nguyện cho tàu điện tử ở ...,akarta cầu nguyện cho tàu điện ngầm để giảm hỗ...,3.80
4996,4996,2014,rực trực thăng rơi gần ga đường ray London,Trực thăng rơi tại London,4.00
4997,4997,2014,exico thề ở tổng thống trong cuộc biểu tổng th...,Thống đốc Richardson của New Mexico đã thúc ép...,0.00
4998,4998,2014,Ai Cập bỏ phiếu cho hiến hiến hiến do Hồi giáo...,Chính phán tại các cuộc thăm thăm chỉ trực tiế...,1.40


In [6]:
examples = []
for index, row in df_train.iterrows():
    tmp1 = row['sentence1']
    tmp2 = row['sentence1']
    examples.append(InputExample(texts=[tmp1, tmp2], label=row['score']/5))
examples = examples[1600:]
dataloader = DataLoader(examples, shuffle=True, batch_size=16)

In [7]:
model.fit(train_objectives=[(dataloader, train_loss)], epochs=10)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/213 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from huggingface_hub import notebook_login, HfApi

notebook_login()

In [ ]:
model.save('/content/model/','modelname',create_model_card=True)

In [ ]:
api = HfApi()
api.upload_folder(
    folder_path='/content/model/',
    repo_id='chienpham/modelname',
    repo_type='model',
)